# Transfer Learning 

Transfer learning from MobileNetV2 and iD3.

# Colab Setup

1. link to and refresh Google Drive
1. Set `pwd` to `colab_notebooks/code/` and import `tensorflow`
1. Check if low or high RAM runtime
1. Connect Tensorflow to Hardware Accelerator, either
  1. GPU, or
  2. TPU

In [1]:
# 1
from google.colab import drive
#drive.mount('/content/drive')
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
# 2
!pwd
%cd drive/MyDrive/colab_notebooks/code/

import sys
import os
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

/content
/content/drive/MyDrive/colab_notebooks/code
Tensorflow version 2.3.0


In [3]:
# 3 
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.4 gigabytes of available RAM

You are using a high-RAM runtime!


In [4]:
# 4.1 GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

device_name = tf.test.gpu_device_name()
print(device_name)
tf.device(device_name)

Tue Dec 15 13:52:37 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# 4.2 TPU
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
#tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
tpu_strategy = tf.distribute.TPUStrategy(tpu)

# MobileNetV2 Pre Trained (2D)

In [ ]:
import tensorflow_hub as hub
from data_mgmt import make_2D_datasets
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
batch_size = 64
img_hw = 160
root_dir = 'data'

[train_ds, test_ds] = make_2D_datasets(batch_size, img_hw, root_dir)
input_shape = (img_hw, img_hw, 3) # does not include batch size

inputs = keras.Input(shape=input_shape)

base_model = tf.keras.applications.MobileNetV2(input_shape=input_shape,
                                               include_top=False,
                                               weights='imagenet')

Found 147358 files belonging to 2 classes.
Found 37738 files belonging to 2 classes.


In [ ]:
# DON"T RUN just seeing it work
#image_batch, label_batch = next(iter(train_ds))
#feature_batch = base_model(image_batch)
#print(feature_batch.shape)

# freezing
#base_model.trainable = False

#global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
#feature_batch_average = global_average_layer(feature_batch)
#print(feature_batch_average.shape)

#prediction_layer = tf.keras.layers.Dense(1)
#prediction_batch = prediction_layer(feature_batch_average)
#print(prediction_batch.shape)

In [ ]:
base_model.trainable = False

inputs = tf.keras.Input(shape=(160, 160, 3))
x = layers.experimental.preprocessing.Rescaling(scale=1./127.5, offset=-1)(inputs) # or just 1./255
x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(256)(x)
x = tf.keras.layers.Dropout(rate=0.2)(x)
x = tf.keras.layers.Dense(128)(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
model = tf.keras.Model(inputs, outputs)

base_learning_rate = 0.000005
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=['accuracy', 'AUC'])

model.summary()

Model: "functional_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        [(None, 160, 160, 3)]     0         
_________________________________________________________________
rescaling_6 (Rescaling)      (None, 160, 160, 3)       0         
_________________________________________________________________
mobilenetv2_1.00_160 (Functi (None, 5, 5, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_6 ( (None, 1280)              0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 1280)              0         
_________________________________________________________________
dense_18 (Dense)             (None, 256)               327936    
_________________________________________________________________
dropout_13 (Dropout)         (None, 256)             

In [ ]:
n_epochs = 10
history = model.fit(train_ds,
                    epochs=n_epochs,
                    validation_data=test_ds)

Epoch 1/10
2303/2303 [==============================] - 33591s 15s/step - loss: 0.4206 - accuracy: 0.8425 - auc: 0.6910 - val_loss: 0.2876 - val_accuracy: 0.8962 - val_auc: 0.8432
Epoch 2/10
2303/2303 [==============================] - 449s 195ms/step - loss: 0.3220 - accuracy: 0.8793 - auc: 0.8167 - val_loss: 0.2662 - val_accuracy: 0.9091 - val_auc: 0.8654
Epoch 3/10
2303/2303 [==============================] - 444s 193ms/step - loss: 0.2918 - accuracy: 0.8907 - auc: 0.8520 - val_loss: 0.2582 - val_accuracy: 0.9128 - val_auc: 0.8722
Epoch 4/10
2303/2303 [==============================] - 446s 194ms/step - loss: 0.2763 - accuracy: 0.8955 - auc: 0.8690 - val_loss: 0.2540 - val_accuracy: 0.9137 - val_auc: 0.8769
Epoch 5/10
2303/2303 [==============================] - 448s 194ms/step - loss: 0.2654 - accuracy: 0.8999 - auc: 0.8803 - val_loss: 0.2501 - val_accuracy: 0.9153 - val_auc: 0.8800
Epoch 6/10
2303/2303 [==============================] - 458s 199ms/step - loss: 0.2576 - accuracy: 0

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
model.save('../saved_models/'+'MobileNetV2')

In [ ]:
import csv
from datetime import datetime

loss = history.history['loss']
accr = history.history['accuracy']
val_l = history.history['val_loss']
val_a = history.history['val_accuracy']
#auc = ??
file_contents = np.array([loss, accr, val_l, val_a]).T
filename = '../visualization/plotting_data/'+'MobileNetV2'+'_training_history.csv'
file = open(filename, 'w+', newline ='')   
with file:
    write = csv.writer(file) 
    write.writerows(file_contents)

# 3D Data Setup (run only once)


In [ ]:
from data_mgmt import make_generators 

In [ ]:
batch_size = 32
img_hw = 160
n_frames = 16
root_dir = 'data'

make_generators(batch_size, img_hw, n_frames, root_dir)

packing 3D arrays...
1/4
2/4
3/4
4/4


# iD3 Pre Trained (3D)

Hara et al. Also called R3D in de Lima.

In [5]:
import tensorflow_hub as hub
from data_mgmt import make_generators224

batch_size = 32
img_hw = 160
n_frames = 16 
root_dir = 'data'
id3_input_shape = (None, 224, 224, 3)

[training_generator, testing_generator] = make_generators224(batch_size, img_hw, n_frames, root_dir)

handle = 'https://tfhub.dev/deepmind/i3d-kinetics-400/1'
do_fine_tuning = False

print("Building model with", handle)
model = tf.keras.Sequential([
    # Explicitly define the input shape so the model can be properly
    # loaded by the TFLiteConverter
    tf.keras.layers.InputLayer(input_shape=id3_input_shape),
    
    hub.KerasLayer(handle, trainable=do_fine_tuning),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
    #tf.keras.layers.Dense(2, kernel_regularizer=tf.keras.regularizers.l2(0.0001))
])

model.build((None,)+id3_input_shape)

base_learning_rate = 0.000005
model.compile(
  optimizer=tf.keras.optimizers.SGD(lr=base_learning_rate, momentum=0.9), 
  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
  metrics=['accuracy', 'AUC'])

model.summary()

reading partition and labels from existing data
Building model with https://tfhub.dev/deepmind/i3d-kinetics-400/1
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 400)               12704544  
_________________________________________________________________
dropout (Dropout)            (None, 400)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               102656    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total pa

In [7]:
checkpoint_path = '../saved_models/checkpoints/id3_1.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)


id3_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
n_epochs=10
history = model.fit(training_generator, epochs=n_epochs, verbose=1,
                    validation_data=testing_generator,
                    callbacks = [id3_callback])

model.save('../saved_models/'+'iD3_1')

Epoch 1/10
278/278 [==============================] - ETA: 0s - loss: 0.9516 - accuracy: 0.4668 - auc: 0.5034 
Epoch 00001: saving model to ../saved_models/checkpoints/id3_1.ckpt
278/278 [==============================] - 3871s 14s/step - loss: 0.9516 - accuracy: 0.4668 - auc: 0.5034 - val_loss: 0.6931 - val_accuracy: 0.8556 - val_auc: 0.5000
Epoch 2/10
278/278 [==============================] - ETA: 0s - loss: 0.7447 - accuracy: 0.7752 - auc: 0.5058
Epoch 00002: saving model to ../saved_models/checkpoints/id3_1.ckpt
278/278 [==============================] - 3392s 12s/step - loss: 0.7447 - accuracy: 0.7752 - auc: 0.5058 - val_loss: 0.6931 - val_accuracy: 0.8552 - val_auc: 0.5000
Epoch 3/10
278/278 [==============================] - ETA: 0s - loss: 0.7028 - accuracy: 0.8375 - auc: 0.5025
Epoch 00003: saving model to ../saved_models/checkpoints/id3_1.ckpt
278/278 [==============================] - 3326s 12s/step - loss: 0.7028 - accuracy: 0.8375 - auc: 0.5025 - val_loss: 0.6931 - val_ac

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


INFO:tensorflow:Assets written to: ../saved_models/iD3_1/assets


INFO:tensorflow:Assets written to: ../saved_models/iD3_1/assets


In [9]:
import csv
from datetime import datetime
import numpy as np

loss = history.history['loss']
accr = history.history['accuracy']
auc = history.history['auc']
val_l = history.history['val_loss']
val_a = history.history['val_accuracy']
val_auc=history.history['val_auc']
file_contents = np.array([loss, accr, auc, val_l, val_a, val_auc]).T
filename = '../visualization/plotting_data/'+'iD3_1'+'_training_history.csv'
file = open(filename, 'w+', newline ='')   
with file:
    write = csv.writer(file)
    write.writerows(file_contents)

Love you Marten! You are mighty and strong! May you wake from your slumber refreshed :)